In [ ]:
!pip install snscrape  #package for social network scrape
!pip install nltk    #package for NLP
!pip install preprocessor
!pip install textblob

In [4]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import os
from re import M
from threading import Lock
import concurrent.futures
from concurrent.futures import as_completed
import subprocess
import sys
import glob
import time
import re
import nltk
import preprocessor as p

In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/srishtiagarwal/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# query__ = "(#cancelculture) lang:en until:2022-12-20 since:2022-01-01"   
# tweets_ =[]
# limit=30000
# count=0
# for tweet in sntwitter.TwitterSearchScraper(query__).get_items():
#        #print(vars(tweet))
#        #break
#         count+=1
#         if count==limit:
#          break
#         else:
#          tweets_.append([tweet.content])

In [235]:
queries = []  
query1 = "(#cancelculture) lang:en until:2022-12-20 since:2022-10-01"   
query2 = "(#cancelculture) lang:en until:2022-9-30 since:2022-07-01"   
query3 = "(#cancelculture) lang:en until:2022-06-30 since:2022-04-01"   
query4 = "(#cancelculture) lang:en until:2022-03-31 since:2022-01-01"   
query5 = "(#cancelculture) lang:en until:2021-12-31 since:2021-10-01"   
query6 = "(#cancelculture) lang:en until:2021-9-30 since:2021-07-01"   
query7 = "(#cancelculture) lang:en until:2021-06-30 since:2021-04-01"   
query8 = "(#cancelculture) lang:en until:2021-03-31 since:2021-01-01"    

for i in range(1, 9):
  text = "query"+str(i)
  queries.append(eval(text))  

tweets = []

In [ ]:
queries = []  
query1 = "armie hammer lang:en until:2021-01-31 since:2021-01-15"   
query2 = "armie hammer lang:en until:2021-02-15 since:2021-02-01"   
  

for i in range(1, 3):
  text = "query"+str(i)
  queries.append(eval(text))  

tweets = []

In [ ]:
queries = []  
query1 = "armie hammer lang:en until:2020-12-31 since:2020-12-01"   
query2 = "armie hammer lang:en until:2020-11-30 since:2020-11-01"   
  

for i in range(1, 3):
  text = "query"+str(i)
  queries.append(eval(text))  

tweets = []

In [236]:
def run_test(query, limit):
   # os.system(f"rm tempTests/{case[13:-4]}.out")                    #removing outputs from previous run if any
    output = ""
    tweets = []
    count = 0
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
      count+=1
      if count==limit:
        break
      else:
        tweets.append([tweet.content])    
    return pd.DataFrame(tweets)

In [ ]:
#def main():
   
tweets=pd.DataFrame()
limit = 20000
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        #max_workers decides the number of threads that will be used in execution
        futures= [executor.submit(run_test, query, limit) for query in queries]
        #futures is calling run_test for each case (every testcase)
        for future in as_completed(futures):    
        #wait for all tasks to complete by getting all results  
            try:  
                data =future.result() 

                tweets= tweets.append(data)
                
                print(f"Finished running ") 
            except Exception as exc:
                print(f"Test Hung \u23F3")
                print(exc)
     

#if __name__ == "__main__":
    #main()


In [ ]:
tweets.columns=['tweet_text']
tweets.count()

In [ ]:
df_orig = pd.DataFrame(tweets, columns=['tweet_text'])
df_orig.count()

In [ ]:
df_orig.head(5)

In [241]:
def preprocess_tweet(sen):
    '''clean the data, leave only 2 or more char long non-stepwords composed of A-Z and a-z only in lowercase'''
    sentence=sen.lower()
    
    #remove RT
    sentence=re.sub('RT @\w+: '," ", sentence)
    
    #remove punctuations and numbers
    sentence = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.

    return sentence

In [242]:
cleaned_tweets = []

for tweet in df_orig['tweet_text']:
    cleaned_tweet = preprocess_tweet(tweet)
    cleaned_tweets.append(cleaned_tweet)

In [ ]:
df_orig['cleaned']=pd.DataFrame(cleaned_tweets)
df_orig.head(5)

In [245]:
#Negative, Positive, Neutral and Compound values

df_orig[['polarity', 'subjectivity']] = df_orig['cleaned'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in df_orig['cleaned'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if comp <= -0.05:
        df_orig.loc[index, 'sentiment'] = "negative"
    elif comp >= 0.05:
        df_orig.loc[index, 'sentiment'] = "positive"
    else:
        df_orig.loc[index, 'sentiment'] = "neutral"
    df_orig.loc[index, 'neg'] = neg
    df_orig.loc[index, 'neu'] = neu
    df_orig.loc[index, 'pos'] = pos
    df_orig.loc[index, 'compound'] = comp

In [ ]:
df_orig.head(5)

In [247]:
#New data frames for sentiments: positive, negative and neutral)

df_orig_negative = df_orig[df_orig["sentiment"]=="negative"]
df_orig_positive = df_orig[df_orig["sentiment"]=="positive"]
df_orig_neutral = df_orig[df_orig["sentiment"]=="neutral"]

In [248]:
#Counting values 

def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [ ]:
#Count_values for sentiment
count_values_in_column(df_orig,"sentiment")